# Movie Reviews Part 2. 

Reference: Jay Alammer http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
<br>
## Overview
<br>

<table border = "5" align = "center">
  <tr>
    <th class="mdc-text-light-green-600" style="text-align:left">
    Movie Reviews Part 1. 
    </th>
    <th class="mdc-text-purple-600">
    Movie Reviews Part 2. 
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      1. Data Processing
    </td>
    <td class="mdc-bg-purple-50">
      BERT and DistilBERT
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      2. Feature Extraction       
    </td>
    <td class="mdc-bg-purple-50">
      1. Tokenization
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Bag of Words, Term Frequency, <p>Inverse Document Frequency
    </td>
    <td class="mdc-bg-purple-50">
      2. Logistic Regression
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      3. Word2Vec4
    </td>
    <td class="mdc-bg-purple-50">
      3. Make Decision
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
     4. Build A Classifier, <p>Identify the sentiment
    </td>
    <td class="mdc-bg-purple-50">
      -
    </td>
  </tr>
    
    
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
     Packages used <p>Pandas 
        <p> Nltk 
        <p> scikit-learn
        <p> Porter Stemmer
    </td>
    <td class="mdc-bg-purple-50" style="text-align:right">
     Packages used <p>numpy
        <p> pandas
        <p> torch
        <p> transformers
        <p> LogisticRegression
    </td>
  </tr>
</table>

<br>

BERT is Google's latest search algorithm as of Oct, 2019. 
    -searchengineland.com
    
BERT - <b>B</b>idirectional <b>E</b>ncoder <b>R</b>epresentations from <b>T</b>ransformers. 

<br>

# A Visual Notebook to Using BERT for the First TIme.ipynb



<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-sentence-classification.png" />

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the text. The text is a list of sentences from film reviews. And we will calssify each sentence as either speaking "positively" about its subject of "negatively".

## Models: Sentence Sentiment Classification
Our goal is to create a model that takes a sentence (just like the ones in our dataset) and produces either 1 (indicating the sentence carries a positive sentiment) or a 0 (indicating the sentence carries a negative sentiment). We can think of it as looking like this:

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence as either positive or negative (1 or 0, respectively).

The data we pass between the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

## Dataset
The dataset we will use in this example is [SST2](https://nlp.stanford.edu/sentiment/index.html), which contains sentences from movie reviews, each labeled as either positive (has the value 1) or negative (has the value 0):


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>

## Installing the transformers library
Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [24]:
!pip install transformers

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [26]:
df = pd.read_csv('/users/kent/jmaharja/olid_training-v3.0.tsv', delimiter='\t', header=None)

For performance reasons, we'll only use 2,000 sentences from the dataset

In [27]:
batch_1 = df[:100]

We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [28]:
batch_1[1].value_counts()

0    68
1    32
Name: 1, dtype: int64

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [29]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

I0313 15:54:03.561112 139871742134080 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /users/kent/jmaharja/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0313 15:54:03.740257 139871742134080 configuration_utils.py:185] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /users/kent/jmaharja/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.587f67ec28c540f4294c9c2ac7dcf7841ff371aeb12cdeb6a17f69da39ad9452
I0313 15:54:03.741357 139871742134080 configuration_utils.py:199] Model config {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [30]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [31]:
max_len = 66
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [32]:
np.array(padded).shape

(100, 126)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [33]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(100, 126)

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [34]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [35]:
features = last_hidden_states[0][:,0,:].numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [36]:
labels = batch_1[1]

In [60]:
features

array([[ 0.06616057, -0.21562211, -0.15828843, ..., -0.11972693,
         0.28065437,  0.4224585 ],
       [-0.07136511, -0.02891544,  0.12300549, ..., -0.2750946 ,
         0.16843088,  0.3923589 ],
       [-0.18865664, -0.10961684, -0.2722153 , ..., -0.1999513 ,
         0.50759685,  0.45119885],
       ...,
       [-0.01398807, -0.11526507,  0.07259701, ..., -0.24692266,
         0.31519768,  0.4025562 ],
       [-0.05027853, -0.1265882 ,  0.01211426, ..., -0.3157476 ,
         0.10798168,  0.5209839 ],
       [-0.08448943, -0.04189255,  0.07106908, ..., -0.23778228,
         0.21741365,  0.43480685]], dtype=float32)

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [37]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

### [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [38]:
 parameters = {'C': np.linspace(0.0001, 100, 20)}
 grid_search = GridSearchCV(LogisticRegression(), parameters)
 grid_search.fit(train_features, train_labels)

 print('best parameters: ', grid_search.best_params_)
 print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 10.526405263157894}
best scrores:  0.76


We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [39]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

<img src="http://127.0.0.1:4000/images/distilBERT/bert-training-logistic-regression.png" />

## Evaluating Model #2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [40]:
lr_clf.score(test_features, test_labels)

0.72

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [43]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.480 (+/- 0.34)


So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.

In [107]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

input_ids = torch.tensor(tokenizer.encode("4 out of 10 British people are basically full-on racists. 4 out of 10 voters vote for the Conservatives.", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)

last_hidden_states = outputs[0]

I0313 16:49:46.129576 139871742134080 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /users/kent/jmaharja/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0313 16:49:46.355584 139871742134080 configuration_utils.py:185] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /users/kent/jmaharja/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.587f67ec28c540f4294c9c2ac7dcf7841ff371aeb12cdeb6a17f69da39ad9452
I0313 16:49:46.356597 139871742134080 configuration_utils.py:199] Model config {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label

In [108]:
last_hidden_states.shape

torch.Size([1, 26, 768])

In [109]:
train_features.shape, train_labels.shape, last_hidden_states.shape


((75, 768), (75,), torch.Size([1, 26, 768]))

In [110]:
tst=last_hidden_states[0,0,:].detach().numpy()
tst.shape

(768,)

In [111]:
temp = np.array(tst).reshape((1, -1))
temp

array([[-8.56884643e-02, -1.40881404e-01, -8.35748464e-02,
        -2.47168854e-01, -5.37201345e-01,  1.89122230e-01,
         4.67723399e-01,  4.03942049e-01, -2.37177238e-01,
        -1.25505090e-01, -1.18157409e-01, -1.68360531e-01,
        -3.33929718e-01,  2.21704654e-02,  1.16631716e-01,
         3.53588223e-01, -2.39305213e-01,  3.84323508e-01,
        -7.29244202e-02,  9.56797749e-02,  1.33756787e-01,
         1.39616430e-02,  9.20621306e-02,  3.41346949e-01,
         3.70616391e-02, -7.14157149e-02,  5.50542593e-01,
        -2.95712829e-01, -2.14483708e-01, -1.64961711e-01,
         4.38891239e-02,  4.45136309e-01, -2.91847855e-01,
        -2.03525245e-01,  3.50762457e-01, -7.72072375e-02,
         1.03549942e-01, -1.92743212e-01,  3.15360934e-01,
         1.98491514e-01, -1.58912867e-01,  8.01379234e-03,
         2.25487977e-01,  3.49239022e-01,  2.21860260e-02,
        -1.10087276e-01, -2.92861319e+00,  2.57292271e-01,
        -1.09352916e-01, -1.17407613e-01, -8.39532316e-0

In [113]:
lr_clf.predict(temp)[0]

0